# Learning sequence of skills
Ally agent is able to create the sequence of skills based on the provided input/output samples.
In the example below, we ask agent to build two skills from scratch with the following requirements:

1. First skill get's nutrients `"category"` name as input and should produce the output similar to what we specify in `"entities"` (for example, list of common nutrients based on provided category)

2. Second skill gets the output of the first skill (`"entities"`) and generate the text, using the examples provided in the ground truth.

In other words, agent learns how to perform the data generation pipeline like `"category"` --> `"entities"` --> `"description"`. 
You can adjust to your specific use case

In [1]:
import sys
sys.path.append('../')

from ally.skills.collection.tg import TextGenerationSkill



from ally.runtimes.openai import OpenAIRuntime
from app.core.settings import settings

from ally.environments.base import StaticEnvironment

from ally.skills.skillset import LinearSkillSet
from ally.agents.base import Agent
import pandas as pd


openai_runtime = OpenAIRuntime(
    verbose=True,
    api_key=settings.openai_api_key,
    gpt_model_name="gpt-3.5-turbo",
    max_tokens=256,
  )


agent = Agent(
    
    # Require agent to learn sequence of two skills
    skills=LinearSkillSet(
      skills=[
        TextGenerationSkill(
          name="skill_0", 
          instruction_template="...",
          input_template="input: {category}",
          output_template=[{
            "name": "entities",
			      "description": "the output of skill_0",
          }]
        ),
        TextGenerationSkill(
          name="skill_1", 
          instruction_template="...",
          input_template="input: {entities}",
          output_template=[{
            "name": "text",
			      "description": "the output of skill_1",
          }]
        )
      ],
    ),
    runtimes={
      'openai': openai_runtime
	  },
    teacher_runtimes={
      'openai': openai_runtime
	  },
    
    # provide ground truth demonstration in environment
    environment=StaticEnvironment(
        df=pd.DataFrame(
            [{
              "category": "Macronutrients",
              "entities": "Carbohydrates, Proteins, Fats",
              "text": "Carbohydrates provide quick energy, proteins are essential for muscle repair and growth, and fats are vital for long-term energy storage and cell function."
            }, {
              "category": "Vitamins",
              "entities": "Vitamin A, Vitamin C, Vitamin D",
              "text": "Vitamin A is crucial for good vision and a healthy immune system, Vitamin C helps in the repair of tissues and the enzymatic production of certain neurotransmitters, and Vitamin D is essential for strong bones and teeth as it helps the body absorb calcium."
            }, {
              "category": "Minerals",
              "entities": "Calcium, Iron, Magnesium",
              "text": "Calcium is necessary for maintaining healthy bones and teeth, Iron is crucial for making red blood cells and transporting oxygen throughout the body, and Magnesium plays a role in over 300 enzyme reactions in the human body, including the metabolism of food, synthesis of fatty acids and proteins, and the transmission of nerve impulses."
            }]
        ),
        ground_truth_columns={
            'entities': 'entities',
            'text': 'text'
        },
        matching_function='fuzzy',
        matching_threshold=0.9
    ),
)
    

In [2]:
agent.learn(learning_iterations=5)

=> Iteration #0: Getting feedback, analyzing and improving ...

Applying skill: skill_0

['category']

  0%|          | 0/3 [00:00<?, ?it/s]

{'category': 'Macronutrients', 'entities': 'Carbohydrates, Proteins, Fats', 'text': 'Carbohydrates provide quick 
energy, proteins are essential for muscle repair and growth, and fats are vital for long-term energy storage and 
cell function.'}

...

{'category': 'Vitamins', 'text': 'Vitamin A is crucial for good vision and a healthy immune system, Vitamin C helps
in the repair of tissues and the enzymatic production of certain neurotransmitters, and Vitamin D is essential for 
strong bones and teeth as it helps the body absorb calcium.', 'entities': 'carbohydrates, proteins, fats'}

100%|██████████| 3/3 [00:06<00:00,  2.15s/it]

{'category': 'Minerals', 'text': 'Calcium is necessary for maintaining healthy bones and teeth, Iron is crucial for
making red blood cells and transporting oxygen throughout the body, and Magnesium plays a role in over 300 enzyme 
reactions in the human body, including the metabolism of food, synthesis of fatty acids and proteins, and the 
transmission of nerve impulses.', 'entities': 'carbohydrates, proteins, fats'}

100%|██████████| 3/3 [02:24<00:00, 48.25s/it]


KeyboardInterrupt: 

In [3]:
predictions = agent.run(pd.DataFrame([
    ['Trace Minerals'],
    ['Water-Soluble Vitamins'],
    ['Fatty Acids']
], columns=['category']))

Applying skill: skill_0

['category']

  0%|          | 0/3 [00:00<?, ?it/s]

{'category': 'Trace Minerals'}

 67%|██████▋   | 2/3 [00:03<00:01,  1.69s/it]

...

 67%|██████▋   | 2/3 [00:02<00:01,  1.34s/it]

{'category': 'Water-Soluble Vitamins', 'entities': 'carbohydrates, proteins, fats'}

100%|██████████| 3/3 [00:05<00:00,  2.02s/it]

{'category': 'Fatty Acids', 'entities': 'carbohydrates, proteins, fats'}

100%|██████████| 3/3 [00:08<00:00,  2.69s/it]


In [4]:
predictions

,category,entities,text
0,Trace Minerals,"carbohydrates, proteins, fats","carbohydrates, proteins, fats"
1,Water-Soluble Vitamins,"carbohydrates, proteins, fats","carbohydrates, proteins, fats"
2,Fatty Acids,"carbohydrates, proteins, fats","carbohydrates, proteins, fats"
